In [1]:
a1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

In [2]:
import math

def init_computer(code, inputs):
    return {
        'mem': code.copy(),
        'mem_size': len(code),
        'extend_mem' : {},
        'inst': 0,
        'rel': 0,
        'inputs': inputs.copy(),
        'outputs': [],
        'halt': False,
        'needs_input': False
    }

def read_mem(computer, pos):
    if(pos > computer['mem_size']):
        if(pos in computer['extend_mem']):
            return computer['extend_mem'][pos]
        else:
            return 0
    else:
        return computer['mem'][pos]

def write_mem(computer, pos, val):
    if(pos > computer['mem_size']):
        computer['extend_mem'][pos] = val
    else:
        computer['mem'][pos] = val

def run(computer):
    code_size = len(computer['mem'])
    i = computer['inst']
    outputs = []
    op_info = {1:3, 2:3, 3:1, 4:1, 5:2, 6:2, 7:3, 8:3, 9:1, 99:0}
    while(True):
        op = read_mem(computer, i)
        opcode = op % 100
        if(not(opcode in op_info)):
            print("error unknown opcode %i" % (opcode))
            computer['needs_input'] = False
            break
        a0 = -1
        a1 = -1
        a2 = -1
        jump = False
        if(op_info[opcode] > 0):
            p_mode = (math.floor(op / 100) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a0 = read_mem(computer, i + 1)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a0 = i + 1
            elif( p_mode == 2 ):
                #relative mode
                a0 = read_mem(computer, i + 1) + computer['rel']
        if(op_info[opcode] > 1):
            p_mode = (math.floor(op / 1000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a1 = read_mem(computer, i + 2)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a1 = i + 2
            elif( p_mode == 2 ):
                #relative mode
                a1 = read_mem(computer, i + 2) + computer['rel']
        if(op_info[opcode] > 2):
            p_mode = (math.floor(op / 10000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a2 = read_mem(computer, i + 3)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a2 = i + 3
            elif( p_mode == 2 ):
                #relative mode
                a2 = read_mem(computer, i + 3) + computer['rel']
        if(opcode == 1):
            #add op
            write_mem(computer, a2, read_mem(computer, a0) + read_mem(computer, a1))
        elif(opcode == 2):
            #mult op
            write_mem(computer, a2, read_mem(computer, a0) * read_mem(computer, a1))
        elif(opcode == 3):
            #read op
            if(len(computer['inputs']) == 0):
                computer['needs_input'] = True
                break
            write_mem(computer, a0, computer['inputs'][0])
            computer['inputs'] = computer['inputs'][1:]
        elif(opcode == 4):
            #write op
            outputs.append(read_mem(computer, a0))
        elif(opcode == 5):
            #jump if true op
            if(read_mem(computer, a0) != 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 6):
            #jump if false op
            if(read_mem(computer, a0) == 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 7):
            #check less than op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) < read_mem(computer, a1)) else 0)
        elif(opcode == 8):
            #check equals op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) == read_mem(computer, a1)) else 0)
        elif(opcode == 9):
            #change relative param op
            computer['rel'] = computer['rel'] + read_mem(computer, a0)
        elif(opcode == 99):
            #halt op
            computer['halt'] = True
            computer['needs_input'] = False
            break
        if(not(jump)):
            i = i + op_info[opcode] + 1
        if(i >= code_size):
            print('exiting b/c end of code reached')
            computer['needs_input'] = False
    computer['outputs'] = outputs
    computer['inst'] = i
    
    return computer

def run_amps(code, setting):
    compA = init_computer(code, [setting[0], 0])
    compB = init_computer(code, [setting[1]])
    compC = init_computer(code, [setting[2]])
    compD = init_computer(code, [setting[3]])
    compE = init_computer(code, [setting[4]])

    while(1):
        compA = run(compA)
        a_out = compA['outputs']
        compA['outputs'] = []
        compB['inputs'] = compB['inputs'] + a_out

        compB = run(compB)
        b_out = compB['outputs']
        compB['outputs'] = []
        compC['inputs'] = compC['inputs'] + b_out

        compC = run(compC)
        c_out = compC['outputs']
        compC['outputs'] = []
        compD['inputs'] = compD['inputs'] + c_out

        compD = run(compD)
        d_out = compD['outputs']
        compD['outputs'] = []
        compE['inputs'] = compE['inputs'] + d_out
        compE = run(compE)
        
        if(compE['halt']):
            break
        
        e_out = compE['outputs']
        
        if(len(e_out) == 0):
            if((compA['needs_input'] or compA['halt']) and 
               (compB['needs_input'] or compB['halt']) and 
               (compC['needs_input'] or compC['halt']) and 
               (compD['needs_input'] or compD['halt']) and 
               (compE['needs_input'] or compE['halt'])):
                return "nope: lacking input and/or halted"
        
        compE['outputs'] = []
        compA['inputs'] = compA['inputs'] + e_out
    
    return compE['outputs'][0]



In [3]:
def perm(l):
    if(len(l) < 2):
        return [l]
    
    p = []
    for i in range(len(l)):
        first = l[i]
        rest = l[:i] + l[i+1:]
        for partial_perm in perm(rest):
            p.append([first] + partial_perm)
    return p

def find_optimal(code, options):
    settings = perm(options)
    highest = []
    highest_power = 0
    for setting in settings:
        power_output = run_amps(code, setting)
        if(power_output > highest_power):
            highest_power = power_output
            highest = setting
    return [highest, highest_power]


#part1_code = [3,8,1001,8,10,8,105,1,0,0,21,46,55,68,89,110,191,272,353,434,99999,3,9,1002,9,3,9,1001,9,3,9,102,4,9,9,101,4,9,9,1002,9,5,9,4,9,99,3,9,102,3,9,9,4,9,99,3,9,1001,9,5,9,102,4,9,9,4,9,99,3,9,1001,9,5,9,1002,9,2,9,1001,9,5,9,1002,9,3,9,4,9,99,3,9,101,3,9,9,102,3,9,9,101,3,9,9,1002,9,4,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,99]
#print("part1:")
#print(find_optimal(part1_code, [0,1,2,3,4]))

In [5]:
code = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
computer = init_computer(code, [])
computer = run(computer)
print(computer['outputs'])

[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99]


In [6]:
code = [1102,34915192,34915192,7,4,7,99,0]
computer = init_computer(code, [])
computer = run(computer)
print(computer['outputs'])

[1219070632396864]


In [7]:
print(run(init_computer([104,1125899906842624,99], []))['outputs'])

[1125899906842624]


In [9]:
part1_code = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,1,3,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,904,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1102,31,1,1009,1101,35,0,1006,1102,1,23,1002,1101,0,32,1013,1101,0,37,1003,1101,0,620,1029,1101,0,28,1011,1102,22,1,1016,1102,1,0,1020,1102,1,34,1007,1102,1,417,1026,1102,1,25,1000,1101,27,0,1010,1102,580,1,1025,1102,1,629,1028,1101,20,0,1004,1102,899,1,1022,1101,26,0,1001,1102,410,1,1027,1102,39,1,1018,1101,0,30,1008,1101,0,38,1014,1101,1,0,1021,1102,29,1,1017,1101,0,36,1012,1101,585,0,1024,1101,0,21,1005,1101,0,892,1023,1102,1,33,1019,1101,24,0,1015,109,17,1206,3,195,4,187,1105,1,199,1001,64,1,64,1002,64,2,64,109,-7,2108,30,-2,63,1005,63,217,4,205,1105,1,221,1001,64,1,64,1002,64,2,64,109,6,1206,5,233,1106,0,239,4,227,1001,64,1,64,1002,64,2,64,109,-16,1202,9,1,63,1008,63,34,63,1005,63,259,1105,1,265,4,245,1001,64,1,64,1002,64,2,64,109,8,1207,-2,34,63,1005,63,285,1001,64,1,64,1105,1,287,4,271,1002,64,2,64,109,-4,1207,-3,27,63,1005,63,305,4,293,1105,1,309,1001,64,1,64,1002,64,2,64,109,-1,21107,40,41,9,1005,1012,331,4,315,1001,64,1,64,1105,1,331,1002,64,2,64,109,5,2107,19,-4,63,1005,63,349,4,337,1106,0,353,1001,64,1,64,1002,64,2,64,109,1,1208,-5,20,63,1005,63,371,4,359,1105,1,375,1001,64,1,64,1002,64,2,64,109,-2,21101,41,0,9,1008,1016,41,63,1005,63,397,4,381,1106,0,401,1001,64,1,64,1002,64,2,64,109,25,2106,0,-5,1001,64,1,64,1105,1,419,4,407,1002,64,2,64,109,-30,2102,1,0,63,1008,63,26,63,1005,63,439,1106,0,445,4,425,1001,64,1,64,1002,64,2,64,109,2,2108,32,4,63,1005,63,465,1001,64,1,64,1105,1,467,4,451,1002,64,2,64,109,-11,1201,10,0,63,1008,63,34,63,1005,63,491,1001,64,1,64,1106,0,493,4,473,1002,64,2,64,109,27,21102,42,1,-1,1008,1019,42,63,1005,63,515,4,499,1105,1,519,1001,64,1,64,1002,64,2,64,109,-6,1201,-7,0,63,1008,63,34,63,1005,63,545,4,525,1001,64,1,64,1106,0,545,1002,64,2,64,109,-15,1202,3,1,63,1008,63,23,63,1005,63,567,4,551,1105,1,571,1001,64,1,64,1002,64,2,64,109,33,2105,1,-8,4,577,1106,0,589,1001,64,1,64,1002,64,2,64,109,-19,1208,-4,34,63,1005,63,605,1105,1,611,4,595,1001,64,1,64,1002,64,2,64,109,7,2106,0,8,4,617,1001,64,1,64,1106,0,629,1002,64,2,64,109,-8,1205,9,647,4,635,1001,64,1,64,1106,0,647,1002,64,2,64,109,-12,2107,38,3,63,1005,63,667,1001,64,1,64,1106,0,669,4,653,1002,64,2,64,109,-3,2102,1,10,63,1008,63,34,63,1005,63,695,4,675,1001,64,1,64,1105,1,695,1002,64,2,64,109,14,21108,43,45,4,1005,1015,711,1105,1,717,4,701,1001,64,1,64,1002,64,2,64,109,13,1205,-4,733,1001,64,1,64,1105,1,735,4,723,1002,64,2,64,109,-30,2101,0,9,63,1008,63,37,63,1005,63,761,4,741,1001,64,1,64,1106,0,761,1002,64,2,64,109,17,21102,44,1,1,1008,1012,45,63,1005,63,785,1001,64,1,64,1106,0,787,4,767,1002,64,2,64,109,5,2101,0,-9,63,1008,63,35,63,1005,63,811,1001,64,1,64,1106,0,813,4,793,1002,64,2,64,109,2,21107,45,44,-5,1005,1013,833,1001,64,1,64,1106,0,835,4,819,1002,64,2,64,109,-2,21101,46,0,-6,1008,1010,44,63,1005,63,855,1106,0,861,4,841,1001,64,1,64,1002,64,2,64,109,2,21108,47,47,-8,1005,1010,883,4,867,1001,64,1,64,1106,0,883,1002,64,2,64,109,2,2105,1,3,1001,64,1,64,1106,0,901,4,889,4,64,99,21102,27,1,1,21102,1,915,0,1105,1,922,21201,1,28815,1,204,1,99,109,3,1207,-2,3,63,1005,63,964,21201,-2,-1,1,21102,1,942,0,1105,1,922,21202,1,1,-1,21201,-2,-3,1,21101,0,957,0,1105,1,922,22201,1,-1,-2,1106,0,968,21202,-2,1,-2,109,-3,2106,0,0]
part1_input = [1]
print("part1:")
print(run(init_computer(part1_code, part1_input))['outputs'])

part1:
[3454977209]


In [10]:
part2_input = [2]
print("part2:")
print(run(init_computer(part1_code, part2_input))['outputs'])

part2:
[50120]
